In [199]:
include("./tire_deg.jl")

function calc_racetimes_basic(;t_base:: Union{Float64,Int},
    tot_no_laps:: Union{Float64,Int},
    t_lap_sens_mass:: Float64,
    t_pitdrive_inlap:: Union{Float64,Int},
    t_pitdrive_outlap:: Union{Float64,Int},
    t_pit_tirechange:: Union{Float64,Int},
    pits_aft_finishline:: Bool,
    cold_tire_pen:: Union{Float64,Int},
    p_grid:: Int,
    t_loss_pergridpos:: Union{Float64,Int},
    t_loss_firstlap:: Union{Float64,Int},
    strategy:: Vector{Vector{Any}},
    m_fuel_init:: Union{Float64,Int},
    b_fuel_perlap:: Float64)

    # ------------------------------------------------------------------------------------------------------------------
    # PREPARATIONS -----------------------------------------------------------------------------------------------------
    # ------------------------------------------------------------------------------------------------------------------

    length(strategy) == 0 && error("Start compound information must be provided")
    length(strategy) == 1 && error("There is no pitstop given in the strategy data. Cars must pit at least once")
    0 in [length(x) == 3 for x in strategy] && error("Inserted strategy data does not contain [inlap, compound, age] for all pit stops!")
    
    #make sure strategy pit stops are in sequential order
    pit_order = [x[1] for x in race_strat]
    for x in length(pit_order)-1
        if pit_order[x] >= pit_order[x+1]
            error("The given inlaps are not sorted in a rising order!")
        end
    end

    # ------------------------------------------------------------------------------------------------------------------
    # CONSIDER BASE LAP TIME, FUEL MASS LOSS AND RACE START ------------------------------------------------------------
    # ------------------------------------------------------------------------------------------------------------------

    # set base lap time
    t_laps = vec(ones(1, tot_no_laps) .* t_base)

    # add fuel mass time loss for every lap (considered with fuel mass at start of respective lap)
    t_laps += (m_fuel_init .- b_fuel_perlap .* [0:1:tot_no_laps-1;]) .* t_lap_sens_mass

    #add race start losses
    t_laps[1] += t_loss_firstlap + (p_grid - 1) * t_loss_pergridpos

    # ------------------------------------------------------------------------------------------------------------------
    # CONSIDER TIRE DEGRADATION ----------------------------------------------------------------------------------------
    # ------------------------------------------------------------------------------------------------------------------

    # loop through all the pit stops
    for idx in 1:length(strategy)
        cur_inlap = strategy[idx][1]
    
        # get current stint length
        if idx  < length(strategy)
            len_cur_stint = strategy[idx + 1][1] - cur_inlap
        else
            len_cur_stint = tot_no_laps - cur_inlap
        end

        # get compound until current pitstop
        comp_cur_stint = strategy[idx][2]

        # get tire age at the beginning of this stint
        age_cur_stint = strategy[idx][3]

        # add tire losses (degradation considered on basis of the tire age at the start of a lap)
        t_laps[cur_inlap+1:cur_inlap + len_cur_stint] += tire_deg.calc_tire_degradation(;tire_age_start=age_cur_stint,stint_length=len_cur_stint,compound=comp_cur_stint)

        # consider cold tires in the first lap of a stint (if inlap is not the last lap of the race)
        if cur_inlap < tot_no_laps
            t_laps[cur_inlap+1] += cold_tire_pen
        end
    end

    # ------------------------------------------------------------------------------------------------------------------
    # CONSIDER PIT STOPS -----------------------------------------------------------------------------------------------
    # ------------------------------------------------------------------------------------------------------------------

    # create array which is not affected by FCY phases afterwards
    t_laps_pit = vec(zeros(1,tot_no_laps))


    # loop through all the pit stops -----------------------------------------------------------------------------------
    for idx in 1:length(strategy)
        cur_inlap = strategy[idx][1]

        # continue if this is the start stint of the strategy list (i.e. no real pit stop)
        if cur_inlap == 0
            continue
        end

        # pit losses (inlap) -------------------------------------------------------------------------------------------
        t_pit_inlap = 0.0

        # consider standstill time
        if !pits_aft_finishline
            t_pit_inlap += t_pit_tirechange
        end
        # pit driving (inlap)
        t_pit_inlap += t_pitdrive_inlap

        # add pit stop loss to t_laps_pit
        t_laps_pit[cur_inlap] += t_pit_inlap

        # pit losses (outlap) ------------------------------------------------------------------------------------------
        # continue if inlap was the last lap of the race and therefore outlap is not driven anymore
        if cur_inlap >= tot_no_laps
            continue
        end

        t_pit_outlap = 0.0

        # consider standstill time
        if pits_aft_finishline
            t_pit_outlap += t_pit_tirechange
        end

        # pit driving (outlap)

        t_pit_outlap += t_pitdrive_outlap

        # add pit stop loss to t_laps_pit
        t_laps_pit[cur_inlap+1] += t_pit_outlap
    end

    # ------------------------------------------------------------------------------------------------------------------
    # CALCULATE LAPWISE RACE TIMES -------------------------------------------------------------------------------------
    # ------------------------------------------------------------------------------------------------------------------

    t_race_lapwise = cumsum(t_laps + t_laps_pit)

    return t_race_lapwise
    #return t_race_lapwise[end]
end


calc_racetimes_basic (generic function with 2 methods)

In [200]:
race_strat = [[0,"medium",5],[5,"soft",10],[50,"hard",5]]

3-element Vector{Vector{Any}}:
 [0, "medium", 5]
 [5, "soft", 10]
 [50, "hard", 5]

In [201]:
test1 = calc_racetimes_basic(;t_base=80, tot_no_laps=70, t_lap_sens_mass=.03, t_pitdrive_inlap=3, t_pitdrive_outlap=16, t_pit_tirechange=3, cold_tire_pen=1.0,
pits_aft_finishline= true, p_grid=5, t_loss_firstlap=10, t_loss_pergridpos=2, strategy=race_strat, m_fuel_init=110, b_fuel_perlap=1.557)

70-element Vector{Float64}:
  102.89999999999999
  186.80329
  270.70986999999997
  354.61974
  441.5329
  545.6131027074705
  629.7926089612554
  714.0820059380364
  798.4926484714213
  883.0367147133506
  967.7272658314138
 1052.5783100347069
 1137.60487124209
    ⋮
 5485.813471540033
 5567.157581540033
 5648.454981540033
 5729.705671540033
 5810.909651540033
 5892.066921540033
 5973.177481540033
 6054.241331540034
 6135.2584715400335
 6216.228901540034
 6297.152621540034
 6378.029631540034